In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"KG-RAG"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [5]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

llm = Ollama(model="gemma2:2b")

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""
print(len(text))

575


In [6]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
# ## 2nd type of filtered documents
# llm_transformer = LLMGraphTransformer(
#     llm=llm,
#     allowed_nodes=["Person", "Country", "Organization"],
#     allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
# )
# graph_documents = llm_transformer.convert_to_graph_documents(
#     documents
# )

In [7]:
graph = NetworkxEntityGraph()

# Add nodes to the graph
for node in graph_documents[0].nodes:
    graph.add_node(node.id)

# Add edges to the graph
for edge in graph_documents[0].relationships:
    graph._graph.add_edge(
            edge.source.id,
            edge.target.id,
            relation=edge.type,
        )

In [8]:
chain = GraphQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True
)

In [9]:
question = """Who is Marie Curie?"""
chain.run(question)

/home/nikunj/Downloads/IRE/Project/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
Marie Curie 

Full Context:
Marie Curie BORN_IN 1867
Marie Curie WAS_A physicist and chemist
Marie Curie WON_NOBEL_PRIZE first woman to win a Nobel Prize
Marie Curie WINS_SECOND_NOBEL_PRIZE first person to win a Nobel Prize twice
Marie Curie WINS_NOBEL_PRIZE_IN_TWO_FIELDS only person to win a Nobel Prize in two scientific fields
Marie Curie WAS_PROFESSOR University of Paris

> Finished chain.


'Marie Curie was a physicist and chemist who was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and only person to win a Nobel Prize in two scientific fields. She also served as a professor at the University of Paris. \n'